In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime, timedelta
import os
import openpyxl
import editpyxl
import excel2img
import xlwt
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')
import numpy as np

#REad the input files

wb1=pd.read_excel('CCR_Part1_Combined.xlsx',sheet_name="Utrancell_Parameters")
wb2=pd.read_excel("CDD_Part1_Combined.xlsx",sheet_name="CELL_PARAMETERS")
wb3=pd.read_excel('LLD_Part1_Combined.xlsx',sheet_name="EutranCellFDD")
wb4=pd.read_excel('CCR_Part1_Combined.xlsx',sheet_name="BB Parameters")

#Step-1 - Copying Data Form "LLD_Part1_Combined" file to " Qatar Ooredoo SRS Template"


wb3["userLabel"]=wb3["userLabel"].astype(str)
wb3["siteid"]=wb3["userLabel"].apply(lambda x: x.split("_")[0])
wb3["site_name"]=wb3["userLabel"].apply(lambda x: x.split("_")[1])
wb3['Tech']="LTE"
#wb3['azimuth']=0
wb3['band']=""
wb3['rnc_id']=""

#Renaming as per the reqd format.

wb3["SITE ID"]=wb3["siteid"]
wb3["CELL OR SECTOR ID"]=wb3["Cell Name"]
wb3["CELL NUMBER OR CID"]=wb3["cellId"]
wb3["SITE NAME (OPTIONAL)"]=wb3["site_name"]
wb3["TECHNOLOGY"]=wb3["Tech"]
wb3["LATITUDE"]=wb3["Lattitude"]
wb3["LONGITUDE"]=wb3["Longitude"]
wb3["FREQUENCY OR FREQUENCY CHANNEL"]=wb3["earfcndl"]
wb3["FREQUENCY BAND"]=wb3["band"]
wb3["eNodeB ID"]=wb3["eNBId"]
wb3["RNC ID"]=wb3["rnc_id"]
wb3["LAC"]=wb3["tac"]
wb3["PSC_PCI"]=wb3[" Physical Cell ID (3xGroup ID+Sub ID)"]
#wb3["LAC"]=wb3["tac"]

lld=wb3[['SITE ID','CELL OR SECTOR ID','CELL NUMBER OR CID','SITE NAME (OPTIONAL)',"TECHNOLOGY","LATITUDE","LONGITUDE",'FREQUENCY OR FREQUENCY CHANNEL','FREQUENCY BAND','eNodeB ID','RNC ID',"LAC","PSC_PCI"]]

#Step-2 - Copying Data Form "CCR_Part1_Combined" file to " Qatar Ooredoo SRS Template"


wb1["Site ID"]=wb1["Site ID"].astype(str)
wb1["siteid"]=wb1["Site ID"].apply(lambda x:x.split("_")[0])
wb1['tech']="WCDMA"
wb1['band']=""
wb1['eNBId']=""

wb4['siteid']=wb4["RBS Name"].apply(lambda x:x.split("_")[0])
azi=wb4[['siteid','beamDirection']]
wb_az=pd.merge(wb1,azi,on=['siteid'])

wb_az["SITE ID"]=wb_az["siteid"]
wb_az["CELL OR SECTOR ID"]=wb_az["UtranCellId"]
wb_az["CELL NUMBER OR CID"]=wb_az["cId"]
wb_az["SITE NAME (OPTIONAL)"]=wb_az["Site Name"]
wb_az["TECHNOLOGY"]=wb_az["tech"]
wb_az["LATITUDE"]=wb_az["Lat"]
wb_az["LONGITUDE"]=wb_az["Long"]
wb_az["FREQUENCY OR FREQUENCY CHANNEL"]=wb_az["uarfcnDl"]
wb_az["FREQUENCY BAND"]=wb_az["band"]
wb_az["eNodeB ID"]=wb_az["eNBId"]
wb_az["RNC ID"]=wb_az["RncId"]
wb_az["LAC"]=wb_az["lac"]
wb_az["PSC_PCI"]=wb_az["primaryScramblingCode"]
wb_az["azimuth"]=wb_az["beamDirection"]

ccr=wb_az[['SITE ID','CELL OR SECTOR ID','CELL NUMBER OR CID','SITE NAME (OPTIONAL)',"TECHNOLOGY","LATITUDE","LONGITUDE","azimuth"\
,'FREQUENCY OR FREQUENCY CHANNEL','FREQUENCY BAND','eNodeB ID','RNC ID',"LAC","PSC_PCI"]]

azi['SITE ID']=azi['siteid']
azi['azimuth']=azi['beamDirection']
azi=azi.drop(['siteid','beamDirection'],axis=1)

lld_1=pd.merge(lld,azi,on=['SITE ID'])

#Step-3 - Copying Data Form "CDD_Part1_Combined" file to " Qatar Ooredoo SRS Template"

wb2["TECHNOLOGY"]="GSM"
wb2["FREQUENCY BAND"]=""
wb2["RNC ID"]=""
wb2["PSC_PCI"]= ""

wb2["SITE ID"]=wb2["Site Code"]                      
wb2["CELL OR SECTOR ID"]=wb2["Cell Name"]          
wb2["CELL NUMBER OR CID"]=wb2["CI"]                 
wb2["SITE NAME (OPTIONAL)"]=wb2["Site Name"]         
#wb2["TECHNOLOGY"]=wb2["tech"]                        
#wb2["LATITUDE"]=wb2["Lat"]                           
#wb2["LONGITUDE"]=wb2["Long"]                         
wb2["FREQUENCY OR FREQUENCY CHANNEL"]=wb2["BCCHNO"]
wb2["FREQUENCY BAND"]=""                    
wb2["eNodeB ID"]=""                        
#wb2["RNC ID"]=wb2["RncId"]                           
#wb2["LAC"]=wb2["LAC"]                                
        
#wb2["AZIMUTH"]=wb2["beamDirection"]    

cdd=wb2[['SITE ID','CELL OR SECTOR ID','CELL NUMBER OR CID','SITE NAME (OPTIONAL)',"TECHNOLOGY"\
,'FREQUENCY OR FREQUENCY CHANNEL','FREQUENCY BAND','eNodeB ID','RNC ID',"LAC","PSC_PCI"]]

#Taking reference of lat/Long and azimuth from ccr.

ccr_ref=ccr[['SITE ID','LATITUDE','LONGITUDE','azimuth']]
cdd_1=pd.merge(cdd,ccr_ref,on=['SITE ID'])

#Removing duplicate rows from all the three dataframe.

cdd_1=cdd_1.drop_duplicates(keep='first')
lld_1=lld_1.drop_duplicates(keep='first')
ccr=ccr.drop_duplicates(keep='first')

#Appending all the three df
final=ccr.append(lld_1,ignore_index=True)
SRS=final.append(cdd_1,ignore_index=True)
#SRS.to_excel("output.xlsx")

SRS['azimuth']=SRS['azimuth'].astype(str)
SRS['azimuth']=SRS['azimuth'].apply(lambda x: '0/0/0' if x=='0' else x)
#Fetching azimuth based on cell name

SRS_01=pd.DataFrame([])
SRS_02=pd.DataFrame([])
SRS_03=pd.DataFrame([])
SRS_04=pd.DataFrame([])
SRS['CELL OR SECTOR ID']=SRS['CELL OR SECTOR ID'].astype(str)
for i in SRS['CELL OR SECTOR ID']:
    #print(i)
    if "_01" in str(i):
        #print(i.index)
        SRS1=SRS[SRS['CELL OR SECTOR ID']==i]
        SRS_01=SRS_01.append(SRS1)
        #print(SRS)
        SRS_01['AZIMUTH']=SRS_01['azimuth'].apply(lambda x:x.split("/")[0])
        
    elif "_02" in str(i) :
        SRS2=SRS[SRS['CELL OR SECTOR ID']==i]
        SRS_02=SRS_02.append(SRS2)
        SRS_02['AZIMUTH']=SRS_02['azimuth'].apply(lambda x:x.split("/")[1])
        #print(SRS)
        #print(SRS['AZIMUTH'])
    elif "_03" in str(i):
        SRS3=SRS[SRS['CELL OR SECTOR ID']==i]
        SRS_03=SRS_03.append(SRS3)
        SRS_03['AZIMUTH']=SRS_03['azimuth'].apply(lambda x:x.split("/")[2])
    elif "_04" in str(i):
        
        SRS4=SRS[SRS['CELL OR SECTOR ID']==i]
        SRS_04=SRS_04.append(SRS4)
        #try:
        SRS_04['AZIMUTH']=SRS_04['azimuth'].apply(lambda x:x.rsplit("/",1)[1])
        #except Exception:
            #pass 
SRS_V1=pd.concat([SRS_01, SRS_02,SRS_03,SRS_04], ignore_index=True)

GSM=SRS[SRS['TECHNOLOGY']=='GSM']

SRS_05=pd.DataFrame([])
SRS_06=pd.DataFrame([])
SRS_07=pd.DataFrame([])
SRS_08=pd.DataFrame([])
GSM['CELL OR SECTOR ID']=GSM['CELL OR SECTOR ID'].astype(str)
for i in GSM['CELL OR SECTOR ID']:
    #print(i)
    if "A" in str(i):
        #print(i.index)
        SRS1=GSM[GSM['CELL OR SECTOR ID']==i]
        SRS_05=SRS_05.append(SRS1)
        #print(SRS)
        SRS_05['AZIMUTH']=SRS_05['azimuth'].apply(lambda x:x.split("/")[0])
        
    elif "B" in str(i) :
        SRS2=GSM[GSM['CELL OR SECTOR ID']==i]
        SRS_06=SRS_06.append(SRS2)
        SRS_06['AZIMUTH']=SRS_06['azimuth'].apply(lambda x:x.split("/")[1])
        #print(SRS)
        #print(SRS['AZIMUTH'])
    elif "C" in str(i):
        SRS3=GSM[GSM['CELL OR SECTOR ID']==i]
        SRS_07=SRS_07.append(SRS3)
        SRS_07['AZIMUTH']=SRS_07['azimuth'].apply(lambda x:x.split("/")[2])
    elif "D" in str(i):
        
        SRS4=GSM[GSM['CELL OR SECTOR ID']==i]
        SRS_08=SRS_08.append(SRS4)
        #try:
        SRS_08['AZIMUTH']=SRS_08['azimuth'].apply(lambda x:x.rsplit("/",1)[1])
        #except Exception:
            #pass 
            
SRS_V2=pd.concat([SRS_05, SRS_06,SRS_07,SRS_08], ignore_index=True)
SRS_FINAL=pd.concat([SRS_V1, SRS_V2], ignore_index=True)
SRS_FINAL=SRS_FINAL.drop(['azimuth'],axis=1)

SRS_FINAL=SRS_FINAL[['SITE ID','CELL OR SECTOR ID','CELL NUMBER OR CID','SITE NAME (OPTIONAL)',"TECHNOLOGY"\
,'LATITUDE','LONGITUDE','AZIMUTH','FREQUENCY OR FREQUENCY CHANNEL','FREQUENCY BAND','eNodeB ID','RNC ID',"LAC","PSC_PCI"]]

SRS_FINAL['CELL OR SECTOR ID']=SRS_FINAL['CELL OR SECTOR ID'].replace(" ","")

#Checking if lat/long are interchanged. Correcting it if reqd.
LATITUDE=[]
i=0
for lat in  SRS_FINAL['LATITUDE']:
    
    if lat<20 or lat>30:
        #print(i)
        lat=SRS_FINAL['LONGITUDE'][i]
        LATITUDE.append(lat)
    else:
        lat=lat
        #print(lat)
        LATITUDE.append(lat)
    i=i+1
LAT=pd.DataFrame(LATITUDE)

LONGITUDE=[]
i=0
for long in  SRS_FINAL['LONGITUDE']:
    
    if long<45 or long>55:
       # print(i)
        long=SRS_FINAL['LATITUDE'][i]
        LONGITUDE.append(long)
    else:
        long=long
        #print(lat)
        LONGITUDE.append(long)
    i=i+1
LONG=pd.DataFrame(LONGITUDE)
SRS_FINAL['LONGITUDE']=LONG
SRS_FINAL['LATITUDE']=LAT

SRS_FINAL=SRS_FINAL.drop_duplicates(keep='first')

SRS_FINAL['INNER_RAD (m)']=15
SRS_FINAL['OUTER_RAD (m)']=250
SRS_FINAL['ANT_BW']=60
SRS_FINAL['INDOOR OR OUTDOOR']=0
SRS_FINAL['GSM OAM BSC OR WCDMA RNC IP (OPTIONAL)']=""
SRS_FINAL['OAM IP for WCDMA eNodeB OR OAM IP for LTE eNodeB (OPTIONAL)']=""
SRS_FINAL['ANTENNA INDEX']=1
SRS_FINAL['TESTING REQUIRED']=0

SRS_FINAL['INNER_RAD (m)']=SRS_FINAL['INNER_RAD (m)'].apply(str)
SRS_FINAL['OUTER_RAD (m)']=SRS_FINAL['OUTER_RAD (m)'].apply(str)
SRS_FINAL['ANT_BW']=SRS_FINAL['ANT_BW'].apply(str)
SRS_FINAL['INDOOR OR OUTDOOR']=SRS_FINAL['INDOOR OR OUTDOOR'].apply(str)
SRS_FINAL['ANTENNA INDEX']=SRS_FINAL['ANTENNA INDEX'].apply(str)
SRS_FINAL['TESTING REQUIRED']=SRS_FINAL['TESTING REQUIRED'].apply(str)





#wb=pd.read_excel("Qatar Ooredoo Swap Project.xlsx",sheet_name="Outdoor Site Data")
#wb6=wb.iloc[:,1:]

#SRS_OQA=pd.concat([wb6,SRS_FINAL],ignore_index=True)
#SRS_OQA=SRS_OQA.drop_duplicates(keep='first')

import shutil

if os.path.isfile('SRS Template.xlsx'):
    os.remove('SRS Template.xlsx')

shutil.copy("Qatar Ooredoo SRS Template.xlsx","SRS Template.xlsx")
book=openpyxl.load_workbook("SRS Template.xlsx")
#sheet = book.get_sheet_by_name("Site Information")

writer=pd.ExcelWriter("SRS Template.xlsx",engine='openpyxl')
writer.book=book
writer.sheets=dict((ws.title,ws) for ws in book.worksheets)
#print(writer.sheets)
#4G
#sitename1.to_excel(writer,"Site Information",startcol=1,startrow=0,header=None,index=False)
SRS_FINAL.to_excel(writer,"Outdoor Site Data",startcol=1,startrow=2,header=None,index=False)
writer.save()
